### Imports und Datei

In [3]:
import osmium as osm
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
import matplotlib.image as mpimg
from urllib.parse import quote
import numpy as np
import os
import json
import requests
from collections import Counter, defaultdict
from matplotlib.ticker import MaxNLocator
from math import radians, cos, sin, asin, sqrt

# Setze den Pfad zum Dataset
dataset_path = os.path.join("Datasets", "unterfranken-latest.osm")
city_name = "Ringheim"   

### Alle Daten aus dem Datensatz auslesen

In [ ]:
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type, lon=None, lat=None, node_ids=None):
        if len(elem.tags) == 0:
            # Für Knoten ohne Tags
            self.osm_data.append([elem_type, 
                                  elem.id, 
                                  elem.visible, 
                                  0, 
                                  None, 
                                  None, 
                                  lon, 
                                  lat, 
                                  node_ids])
        else:
            for tag in elem.tags:
                self.osm_data.append([elem_type, 
                                      elem.id,
                                      elem.visible,
                                      len(elem.tags),
                                      tag.k, 
                                      tag.v, lon, lat, node_ids])

    def node(self, n):
        self.tag_inventory(n, "node", lon=n.location.lon, lat=n.location.lat)

    def way(self, w):
        node_ids = [n.ref for n in w.nodes]  
        self.tag_inventory(w, "way", node_ids=node_ids)

# Daten auslesen
osmhandler = OSMHandler()
osmhandler.apply_file(dataset_path)

# DataFrame erstellen
data_colnames = ['type', 'id', 'visible', 'ntags', 'tagkey', 'tagvalue', 'longitude', 'latitude', 'node_ids']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
df_osm

### Datensatz nach Wegen gefiltert

In [ ]:
ways_filter = df_osm[(df_osm['type'] == 'way') & (df_osm['tagkey'] == 'highway')]
ways_filter

### API-Anfrage für die Längen- und Breitengrade der Städte

In [ ]:
# API-Anfrage um Bounding Box der Stadt zu bekommen
def get_bounding_box(city_name):
    url = f"https://nominatim.openstreetmap.org/search?city={city_name}&format=json"
    headers = {
        "User-Agent": "Bachelorproject/1.0"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        results = response.json()
        for result in results:
             if result.get('osm_type') == 'relation' and 'boundingbox' in result:
                bbox = result['boundingbox']
                return [float(coord) for coord in bbox]  
    raise ValueError("Bounding Box konnte nicht abgerufen werden.")
     
bbox = get_bounding_box(city_name)
south, north, west, east = bbox

df_filtered_nodes = df_osm[
    ((df_osm['latitude'] >= south) & (df_osm['latitude'] <= north)) &
    ((df_osm['longitude'] >= west) & (df_osm['longitude'] <= east))    
]

df_filtered_nodes

### Funktion um Stadt-Daten extra zu speichern

In [6]:
def save_city_data_json(df_filtered, city_name, output_folder="City_data"):
    output_folder = os.path.abspath(output_folder)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_file = os.path.join(output_folder, f"{city_name}.json")
    json_data = []

    try:
        # Durch alle Zeilen im DataFrame iterieren
        for _, row in df_filtered.iterrows():
            element = {}
            element['type'] = row['type']
            element['id'] = row['id']
            element['visible'] = row['visible']
            element['longitude'] = row['longitude']
            element['latitude'] = row['latitude']

            # Tags hinzufügen, wenn vorhanden
            if row['tagkey']:
                element['tags'] = {row['tagkey']: row['tagvalue']}
            else:
                element['tags'] = {}

            # Wenn es ein 'way' ist, füge die node_ids hinzu
            if row['type'] == 'way':
                element['node_ids'] = row['node_ids']

            # Die gesammelten Daten in die Liste hinzufügen
            json_data.append(element)

        # Speichern der JSON-Daten in die Datei
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)

        print(f"Daten für {city_name} wurden in {output_file} gespeichert.")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei: {e}")

### Datensatz nach Städten filtern

In [ ]:
ways_filter = ways_filter[ways_filter['node_ids'].apply(lambda node_ids: any(node in df_filtered_nodes['id'].values for node in node_ids))]

df_filtered = pd.concat([df_filtered_nodes, ways_filter])
df_filtered = df_filtered.fillna("")
save_city_data_json(df_filtered, city_name)
df_filtered

### Datensatz auf nur Kreuzungspunkte reduzieren

In [ ]:
def process_and_save_city_data_json(city_name, input_folder="City_data"):
    filename = f"{city_name}.json"
    input_file = os.path.join(input_folder, filename)
    
    if not os.path.exists(input_file):
        print(f"Datei für {city_name} wurde nicht gefunden.")
        return        

    with open(input_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    df_from_file = pd.json_normalize(json_data)

    # Alle Knoten-IDs aus den Wegen extrahieren
    all_node_ids = []
    for node_ids_list in df_from_file[df_from_file['type'] == 'way']['node_ids']:
        if node_ids_list is not None:
            all_node_ids.extend(node_ids_list)

    node_reference_count = Counter(all_node_ids)

    # Nur Knoten, die in mehr als einem Weg referenziert werden (Kreuzungsknoten)
    nodes_referenced_multiple = {node_id for node_id, count in node_reference_count.items() if count > 1}

    # Nur die Knoten, die Kreuzungsknoten sind
    df_nodes_filtered_multiple = df_from_file[(df_from_file['type'] == 'node') & (df_from_file['id'].isin(nodes_referenced_multiple))]

    # Nur Wege, die Kreuzungsknoten enthalten
    ways_from_file = df_from_file[df_from_file['type'] == 'way']
    ways_from_file = ways_from_file[ways_from_file['node_ids'].apply(lambda node_ids: any(node in nodes_referenced_multiple for node in node_ids))]

    # Knoten und Wege in separaten Listen speichern
    nodes = []
    for _, row in df_nodes_filtered_multiple.iterrows():
        node = {
            "type": "node",
            "id": row['id'],
            "visible": row['visible'],
            "latitude": row['latitude'],
            "longitude": row['longitude'],
            "tags": {}
        }
        # Überprüfen, ob 'tagkey' und 'tagvalue' vorhanden sind
        if 'tagkey' in row and pd.notna(row['tagkey']) and 'tagvalue' in row and pd.notna(row['tagvalue']):
            node["tags"][row['tagkey']] = row['tagvalue']
        nodes.append(node)

    ways = []
    for _, row in ways_from_file.iterrows():  
        way = {
            "type": "way",
            "id": row['id'],
            "visible": row['visible'],
            "node_ids": row['node_ids'],
            "tags": {}
        }
        # Überprüfen, ob 'tagkey' und 'tagvalue' vorhanden sind
        if 'tagkey' in row and pd.notna(row['tagkey']) and 'tagvalue' in row and pd.notna(row['tagvalue']):
            way["tags"][row['tagkey']] = row['tagvalue']
        ways.append(way)

    # Alle Knoten und Wege zusammenstellen
    filtered_data = {"nodes": nodes, "ways": ways}
    print(len(nodes))

    # Speichern der gefilterten Daten zurück in die gleiche JSON-Datei
    output_file = os.path.join(input_folder, filename)
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_data, f, ensure_ascii=False, indent=4)

process_and_save_city_data_json(city_name)

### Graph vor dem Entfernen der Grad-2-Knoten

In [ ]:
def load_city_data_json(city_name, input_folder="City_data"):
    filename = f"{city_name}.json"
    input_file = os.path.join(input_folder, filename)
    
    if not os.path.exists(input_file):
        print(f"Datei für {city_name} wurde nicht gefunden.")
        return None

    with open(input_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def create_graph_from_city_data(city_name, input_folder="City_data"):
    filtered_data = load_city_data_json(city_name, input_folder)
    if filtered_data is None:
        return None

    G = nx.Graph()

    # Knoten zum Graphen hinzufügen
    for node in filtered_data["nodes"]:
        if "latitude" in node and "longitude" in node:
            G.add_node(node['id'], pos=(node['longitude'], node['latitude']))

    # Kanten zum Graphen hinzufügen
    for way in filtered_data["ways"]:
        node_ids_list = way.get('node_ids', [])
        if node_ids_list:
            # Filtere nur die Knoten-IDs, die im Graphen vorhanden sind
            filtered_node_ids_list = [node_id for node_id in node_ids_list if G.has_node(node_id)]
            if len(filtered_node_ids_list) > 1:
                for i in range(len(filtered_node_ids_list) - 1):
                    # Bestimme, ob die Straße gestrichelt sein soll
                    is_dashed_highway = way.get('tags', {}).get('highway') not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']
                    G.add_edge(
                        filtered_node_ids_list[i], filtered_node_ids_list[i + 1],
                        street_name=way.get('tags', {}).get('highway', ''),
                        is_dashed_highway=is_dashed_highway
                    )

    return G

# Visualisierung des Graphen
def plot_graph(city_name, input_folder="City_data"):
    G = create_graph_from_city_data(city_name, input_folder)
    if G is None:
        return

    pos = nx.get_node_attributes(G, 'pos')

    plt.figure(figsize=(12, 8))

    # Normale Kanten zeichnen
    edges_normal = [(u, v) for u, v, d in G.edges(data=True) if not d.get('is_dashed_highway')]
    nx.draw_networkx_edges(G, pos, edgelist=edges_normal, edge_color='gray', width=1.5)

    # Gestrichelte Kanten zeichnen
    edges_dashed = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_dashed_highway')]
    nx.draw_networkx_edges(G, pos, edgelist=edges_dashed, edge_color='gray', width=1.5, style='--')

    node_degrees = dict(G.degree())
    node_color = ['red' if node_degrees[node] >= 3 else 'blue' for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_size=50, node_color=node_color)

    plt.title(f"Graph der Stadt: {city_name}")
    plt.xlabel("Längengrad")
    plt.ylabel("Breitengrad")
    plt.show()

plot_graph(city_name)


### Knoten vom Grad 2 entfernen

#### Funktionen für die Bearbeitung der Grad 2 Knoten

In [10]:
G = nx.Graph()

neighbors_found = {}

# Entfernen von Knoten mit Grad 2 aus den node-ids
def remove_degree_2_nodes(node_ids):
    if node_ids is not None:
        return [node_id for node_id in node_ids if node_id not in degree_2_node_ids]
    return node_ids

# 2 Grad-3+-Nachbarn finden
def find_neighbors_with_degree_3(node_id, visited=None, found_neighbors=None):
    if visited is None:
        visited = set()
    if found_neighbors is None:
        found_neighbors = []

    visited.add(node_id)
    neighbors = list(G.neighbors(node_id))

    for neighbor in neighbors:
        if neighbor not in visited:
            neighbor_degree = G.degree(neighbor)

            # Füge Nachbarn hinzu, wenn er Grad 3 oder höher hat
            if neighbor_degree >= 3:
                found_neighbors.append(neighbor)

                if node_id not in neighbors_found:
                    neighbors_found[node_id] = []
                neighbors_found[node_id].append(neighbor)

                # Stoppe, wenn zwei passende Nachbarn gefunden wurden
                if len(found_neighbors) == 2:
                    return found_neighbors

            # Rekursive Suche fortsetzen, wenn der Nachbar Grad 2 hat
            elif neighbor_degree == 2:
                result = find_neighbors_with_degree_3(neighbor, visited, found_neighbors)
                if len(result) == 2:
                    return result
                
    return found_neighbors

# Aktualisieren der node_ids der Straßen, die Knoten mit Grad 2 enthalten
def update_node_ids(row, degree_2_node_ids):
    node_ids = row['node_ids']
    if node_ids is not None:
        updated_node_ids = list(node_ids)
        for node_id in node_ids:
            if node_id in degree_2_node_ids:
                # Finde Nachbarn des Knotens mit Grad 2
                neighbors = find_neighbors_with_degree_3(node_id)
                if len(neighbors) == 2:
                    neighbor_1, neighbor_2 = neighbors

                    # Finde die zwei Straßen, die den Grad-2-Knoten referenzieren
                    referencing_ways = ways_filter[ways_filter['node_ids'].apply(lambda x: node_id in x)]

                    if len(referencing_ways) == 2:
                        way_1, way_2 = referencing_ways.iloc[0], referencing_ways.iloc[1]

                        # Kopiere die Node-IDs als Listen
                        way_1_node_ids = list(way_1['node_ids']) if isinstance(way_1['node_ids'], list) else way_1['node_ids']
                        way_2_node_ids = list(way_2['node_ids']) if isinstance(way_2['node_ids'], list) else way_2['node_ids']

                        # Füge Nachbarn hinzu, falls sie nicht bereits vorhanden sind
                        if neighbor_1 not in way_2_node_ids:
                            way_2_node_ids.append(neighbor_1)
                        if neighbor_1 not in way_1_node_ids:
                            way_1_node_ids.append(neighbor_1)

                        # Aktualisiere die DataFrame-Zeilen
                        ways_filter.at[way_1.name, 'node_ids'] = way_1_node_ids
                        ways_filter.at[way_2.name, 'node_ids'] = way_2_node_ids

        # Stelle sicher, dass die Rückgabewerte korrekt sind
        return updated_node_ids if updated_node_ids else None
    return node_ids

#### Funktionen für die Bearbeitung der speziellen Grad-2-Knoten

In [11]:
def handle_special_degree_2_nodes(special_nodes, connected_ways, G):
    neighbors_by_special_node = {}

    def find_special_neighbors(node_id, connected_node_ids, visited=None, found_neighbors=None):
        if visited is None:
            visited = set()
        if found_neighbors is None:
            found_neighbors = set()

        visited.add(node_id)
        neighbors = list(G.neighbors(node_id))

        for neighbor in neighbors:
            if neighbor not in visited and neighbor in connected_node_ids:  
                neighbor_degree = G.degree(neighbor)

                # Füge Nachbarn hinzu, wenn er Grad 3 oder höher hat
                if neighbor_degree >= 3:
                    found_neighbors.add(neighbor) 

                    # Stoppe, wenn zwei passende Nachbarn gefunden wurden
                    if len(found_neighbors) == 2:
                        return list(found_neighbors)

                # Rekursive Suche fortsetzen, wenn der Nachbar Grad 2 hat
                elif neighbor_degree == 2:
                    result = find_special_neighbors(neighbor, connected_node_ids, visited, found_neighbors)
                    if len(result) == 2:
                        return list(result)

        return list(found_neighbors)

    # Für jeden speziellen Knoten die Nachbarn durch verbundene Wege finden
    for node_id, ways in connected_ways.items():
        neighbors_by_special_node[node_id] = []
        connected_node_ids = set()
        for way_id in ways:
            way = ways_filter[ways_filter['id'] == way_id]
            if not way.empty:
                connected_node_ids.update(way.iloc[0]['node_ids'])

        # Suche Nachbarn des speziellen Knotens, die Grad 3+ haben und in connected_node_ids liegen
        neighbors = find_special_neighbors(node_id, connected_node_ids)
        if neighbors:
            neighbors_by_special_node[node_id].extend(neighbors)

    return neighbors_by_special_node

# Identifizieren der speziellen Grad-2-Knoten
def identify_special_degree_2_nodes(node_way_mapping):
    special_nodes = {node_id: ways for node_id, ways in node_way_mapping.items() if len(ways) > 2}
    return special_nodes

# Ermitteln der Straßen, die mit speziellen Grad-2-Knoten verbunden sind
def get_connected_ways_for_special_nodes(df_special_nodes, valid_ways, G):
    connected_ways_by_node = {}

    for _, node_row in df_special_nodes.iterrows():
        node_id = node_row['id']
        connected_ways_by_node[node_id] = []
        
        # Finde Wege, die mit diesem speziellen Knoten verbunden sind
        for _, way_row in valid_ways.iterrows():
            way_id = way_row['id']
            node_ids_list = way_row['node_ids']
            
            if node_id in node_ids_list:
                # Prüfen, ob die Knoten des Wegs im Graphen existieren
                for i in range(len(node_ids_list) - 1):
                    u, v = node_ids_list[i], node_ids_list[i + 1]
                    if G.has_edge(u, v):
                        connected_ways_by_node[node_id].append(way_id)
                        break

    return connected_ways_by_node

# Aktualisieren der node_ids für spezielle Knoten
def update_special_node_ids(row, special_nodes, neighbors_by_special_node, connected_ways_for_special_nodes):
    node_ids = row['node_ids']

    if not isinstance(node_ids, list):
        node_ids = []
    
    if node_ids:
        updated_node_ids = list(node_ids)
        
        for node_id in node_ids:
            if node_id in special_nodes:
                neighbors = neighbors_by_special_node.get(node_id, [])
        
                if len(neighbors) == 2:
                    neighbor_1, neighbor_2 = neighbors
                    referencing_ways = connected_ways_for_special_nodes.get(node_id, [])
                    if len(referencing_ways) == 2:
                        way_1_id, way_2_id = referencing_ways[0], referencing_ways[1]
                        
                        way_1 = ways_filter[ways_filter['id'] == way_1_id].iloc[0]
                        way_2 = ways_filter[ways_filter['id'] == way_2_id].iloc[0]

                        way_1_node_ids = list(way_1['node_ids']) if isinstance(way_1['node_ids'], list) else way_1['node_ids']
                        way_2_node_ids = list(way_2['node_ids']) if isinstance(way_2['node_ids'], list) else way_2['node_ids']

                        if neighbor_1 not in way_2_node_ids:
                            way_2_node_ids.append(neighbor_1)
                        if neighbor_1 not in way_1_node_ids:
                            way_1_node_ids.append(neighbor_1)

                        ways_filter.at[way_1.name, 'node_ids'] = way_1_node_ids
                        ways_filter.at[way_2.name, 'node_ids'] = way_2_node_ids
        
        return updated_node_ids if updated_node_ids else None
    
    return node_ids

# Entfernen der speziellen und normalen Knoten
def remove_special_nodes_from_ids(node_ids, special_node_ids):
    if node_ids is not None:
        return [node_id for node_id in node_ids if node_id not in special_node_ids]
    return node_ids

#### Aufruf zum Entfernen der Grad-2-Knoten

In [ ]:
json_file_path = os.path.join('City_data', f'{city_name}.json')
    
with open(json_file_path, 'r') as file:
    data = json.load(file)

nodes_data = data.get('nodes', [])
ways_data = data.get('ways', [])   

nodes_df = pd.DataFrame(nodes_data)    
ways_filter = pd.DataFrame(ways_data)

filtered_data_multiple = pd.concat([nodes_df, ways_filter], ignore_index=True)    

# Extrahieren der Knoten_Ids
existing_node_ids = set(filtered_data_multiple[filtered_data_multiple['type'] == 'node']['id'])

G = nx.Graph()
# Wege durchgehen und Kanten im Graphen hinzufügen
for _, row in ways_filter.iterrows():
    node_ids = row['node_ids']
    if node_ids is not None:
        valid_node_ids = [node_id for node_id in node_ids if node_id in existing_node_ids]
        for i in range(len(valid_node_ids) - 1):
            G.add_edge(valid_node_ids[i], valid_node_ids[i + 1])

# Knoten mit Grad 2 finden
node_degrees = dict(G.degree())
nodes_with_degree_2 = [node_id for node_id, degree in node_degrees.items() if degree == 2]

# Knoten mit Grad 2 in filtered_data_multiple filtern
df_nodes_degree_2 = filtered_data_multiple[(filtered_data_multiple['type'] == 'node') & (filtered_data_multiple['id'].isin(nodes_with_degree_2))]
degree_2_node_ids = set(df_nodes_degree_2['id'])

# Spezielle Knoten mit Grad 2 finden
nodes_degree_2 = [node for node, degree in G.degree() if degree == 2]
node_way_mapping = {node_id: [] for node_id in nodes_degree_2}
for index, way in ways_filter.iterrows():
    node_ids_list = way['node_ids']
    if node_ids_list is not None:
        for node_id in nodes_degree_2:
            if node_id in node_ids_list:
                node_way_mapping[node_id].append(way['id'])
special_nodes = identify_special_degree_2_nodes(node_way_mapping)

# Datensatz für spezielle Knoten
special_node_ids = set(special_nodes.keys()) 
df_special_nodes = filtered_data_multiple[filtered_data_multiple['id'].isin(special_node_ids)]

# Straßen mit speziellen Knoten ermitteln
connected_ways_for_special_nodes = get_connected_ways_for_special_nodes(df_special_nodes, ways_filter, G)

# Nachbarn von speziellen Knoten ermitteln
neighbors_by_special_node = handle_special_degree_2_nodes(df_special_nodes, connected_ways_for_special_nodes, G)

# Aktualisieren die node_ids der speziellen Straßen
filtered_data_multiple['node_ids'] = filtered_data_multiple.apply(lambda row: update_special_node_ids(row, special_nodes, neighbors_by_special_node, connected_ways_for_special_nodes), axis=1)

# Aktualisiere die node_ids der Straßen
filtered_data_multiple['node_ids'] = filtered_data_multiple.apply(lambda row: update_node_ids(row, degree_2_node_ids), axis=1)

# Entfernen der Knoten mit Grad 2 aus den node_ids der Straßen
filtered_data_multiple['node_ids'] = filtered_data_multiple['node_ids'].apply(lambda node_ids: remove_special_nodes_from_ids(node_ids, degree_2_node_ids.union(special_node_ids)))

# Entferne Knoten mit Grad 2 aus dem Datensatz
filtered_data_multiple = filtered_data_multiple[~filtered_data_multiple['id'].isin(degree_2_node_ids.union(special_node_ids))]

# Entferne unverbundene Knoten aus dem Datensatz
connected_nodes = set(G.nodes)
filtered_data_multiple = filtered_data_multiple[(filtered_data_multiple['type'] != 'node') | (filtered_data_multiple['id'].isin(connected_nodes))]

filtered_data_multiple = filtered_data_multiple.fillna("")

updated_data = {
    'nodes': filtered_data_multiple[filtered_data_multiple['type'] == 'node'].to_dict(orient="records"),
    'ways': filtered_data_multiple[filtered_data_multiple['type'] == 'way'].to_dict(orient='records')
}

with open(json_file_path, 'w') as file:
    json.dump(updated_data, file, indent=4)

print(f"Die Daten für die Stadt {city_name} wurden erfolgreich aktualisiert und gespeichert.")

### Graph nach dem Entfernen der 2-Grad-Knoten

In [ ]:
def load_city_data_json(city_name, input_folder="City_data"):
    filename = f"{city_name}.json"
    input_file = os.path.join(input_folder, filename)
    
    if not os.path.exists(input_file):
        print(f"Datei für {city_name} wurde nicht gefunden.")
        return None

    with open(input_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def create_graph_from_city_data(city_name, input_folder="City_data"):
    filtered_data = load_city_data_json(city_name, input_folder)
    if filtered_data is None:
        return None

    G = nx.Graph()

    # Knoten zum Graphen hinzufügen
    for node in filtered_data["nodes"]:
        if "latitude" in node and "longitude" in node:
            G.add_node(node['id'], pos=(node['longitude'], node['latitude']))

    # Kanten zum Graphen hinzufügen
    for way in filtered_data["ways"]:
        node_ids_list = way.get('node_ids', [])
        if node_ids_list:
            # Filtere nur die Knoten-IDs, die im Graphen vorhanden sind
            filtered_node_ids_list = [node_id for node_id in node_ids_list if G.has_node(node_id)]
            if len(filtered_node_ids_list) > 1:
                for i in range(len(filtered_node_ids_list) - 1):
                    # Bestimme, ob die Straße gestrichelt sein soll
                    is_dashed_highway = way.get('tags', {}).get('highway') not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']
                    G.add_edge(
                        filtered_node_ids_list[i], filtered_node_ids_list[i + 1],
                        street_name=way.get('tags', {}).get('highway', ''),
                        is_dashed_highway=is_dashed_highway
                    )

    return G

# Visualisierung des Graphen
def plot_graph(city_name, input_folder="City_data"):
    G = create_graph_from_city_data(city_name, input_folder)
    if G is None:
        return

    pos = nx.get_node_attributes(G, 'pos')

    plt.figure(figsize=(12, 8))

    # Normale Kanten zeichnen
    edges_normal = [(u, v) for u, v, d in G.edges(data=True) if not d.get('is_dashed_highway')]
    nx.draw_networkx_edges(G, pos, edgelist=edges_normal, edge_color='gray', width=1.5)

    # Gestrichelte Kanten zeichnen
    edges_dashed = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_dashed_highway')]
    nx.draw_networkx_edges(G, pos, edgelist=edges_dashed, edge_color='gray', width=1.5, style='--')

    node_degrees = dict(G.degree())
    node_color = ['red' if node_degrees[node] >= 3 else 'blue' for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_size=50, node_color=node_color)

    plt.title(f"Graph der Stadt: {city_name}")
    plt.xlabel("Längengrad")
    plt.ylabel("Breitengrad")
    plt.show()

plot_graph(city_name)

### Haversine Formel für die Längenberechnung

In [5]:
def haversine(lon1, lat1, lon2, lat2):
    # Dezimalzahlen in Radianten umwandeln
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine Formel
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius der Erde in Kilometer
    return c * r * 1000

### Knoten zusammenfügen, die nah beieinander stehen

In [ ]:
# Löschen der Knoten, die durch anderen ersetzt wurde
def remove_replaced_nodes(df_nodes, node_merge_dict):
    replaced_node_ids = set(node_merge_dict.keys()) - set(node_merge_dict.values())
    return df_nodes[~df_nodes['id'].isin(replaced_node_ids)]

# Aktualisiere node_ids
def update_node_ids_merged(node_ids, node_merge_dict):
    if node_ids is not None and isinstance(node_ids, list):
        updated_node_ids = [node_merge_dict.get(node_id, node_id) for node_id in node_ids]
        # Entferne doppelte Knoten, falls sie durch das Zusammenführen entstanden sind
        unique_node_ids = list(dict.fromkeys(updated_node_ids))
        return unique_node_ids
    return node_ids

# Zusammenfügen der Knoten, die nah beieinander stehen
def merge_within_distance(df_nodes, max_distance=10):
    merged_nodes = []
    node_merge_dict = {}
    seen = set()

    for i, node_1 in df_nodes.iterrows():
        for j, node_2 in df_nodes.iterrows():
            # Vergleich gleicher Knoten verhindern
            if node_1['id'] == node_2['id'] or node_1['id'] in seen or node_2['id'] in seen:
                continue

            lon1, lat1 = node_1['longitude'], node_1['latitude']
            lon2, lat2 = node_2['longitude'], node_2['latitude']

            distance = haversine(lon1, lat1, lon2, lat2)

            if distance < max_distance:
                merged_node = {
                    'id': node_1['id'],
                    'longitude': (lon1 + lon2) / 2,
                    'latitude': (lat1 + lat2) / 2
                }
                merged_nodes.append(merged_node)

                # Knoten als gesehen markiert
                seen.add(node_1['id'])
                seen.add(node_2['id'])

                node_merge_dict[node_2['id']] = node_1['id']

    return merged_nodes, node_merge_dict

def process_city_data(city_name):
    json_file_path = os.path.join('City_data', f'{city_name}.json')
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    nodes_data = data.get('nodes', [])
    ways_data = data.get('ways', [])

    df_nodes = pd.DataFrame(nodes_data)
    df_ways = pd.DataFrame(ways_data)

    merged_nodes, node_merge_dict = merge_within_distance(df_nodes)

    # Wege aktualisieren
    df_ways['node_ids'] = df_ways['node_ids'].apply(
        lambda x: update_node_ids_merged(x, node_merge_dict)
    )

    # Ehemalige Knoten entfernen
    df_nodes_updated = remove_replaced_nodes(df_nodes, node_merge_dict)

    # Zusammenführen der aktualisierten Knoten und Wege
    updated_nodes_data = df_nodes_updated.to_dict(orient="records")
    updated_ways_data = df_ways.to_dict(orient="records")

    # Aktualisierte Daten in die Datei zurückschreiben
    updated_data = {
        'nodes': updated_nodes_data,
        'ways': updated_ways_data
    }

    # Speichern der Datei
    with open(json_file_path, 'w') as file:
        json.dump(updated_data, file, indent=4)

    print(f"Die Daten für die Stadt {city_name} wurden erfolgreich aktualisiert und gespeichert.")

process_city_data(city_name)

### Graph nach Zusammenfügen der Knoten die beieinander stehen

In [ ]:
def load_city_data_json(city_name, input_folder="City_data"):
    filename = f"{city_name}.json"
    input_file = os.path.join(input_folder, filename)
    
    if not os.path.exists(input_file):
        print(f"Datei für {city_name} wurde nicht gefunden.")
        return None

    with open(input_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def create_graph_from_city_data(city_name, input_folder="City_data"):
    filtered_data = load_city_data_json(city_name, input_folder)
    if filtered_data is None:
        return None

    G = nx.Graph()

    # Knoten zum Graphen hinzufügen
    for node in filtered_data["nodes"]:
        if "latitude" in node and "longitude" in node:
            G.add_node(node['id'], pos=(node['longitude'], node['latitude']))

    # Kanten zum Graphen hinzufügen
    for way in filtered_data["ways"]:
        node_ids_list = way.get('node_ids', [])
        if node_ids_list:
            # Filtere nur die Knoten-IDs, die im Graphen vorhanden sind
            filtered_node_ids_list = [node_id for node_id in node_ids_list if G.has_node(node_id)]
            if len(filtered_node_ids_list) > 1:
                for i in range(len(filtered_node_ids_list) - 1):
                    # Bestimme, ob die Straße gestrichelt sein soll
                    is_dashed_highway = way.get('tags', {}).get('highway') not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']
                    G.add_edge(
                        filtered_node_ids_list[i], filtered_node_ids_list[i + 1],
                        street_name=way.get('tags', {}).get('highway', ''),
                        is_dashed_highway=is_dashed_highway
                    )

    return G

# Visualisierung des Graphen
def plot_graph(city_name, input_folder="City_data"):
    G = create_graph_from_city_data(city_name, input_folder)
    if G is None:
        return

    pos = nx.get_node_attributes(G, 'pos')

    plt.figure(figsize=(12, 8))

    # Normale Kanten zeichnen
    edges_normal = [(u, v) for u, v, d in G.edges(data=True) if not d.get('is_dashed_highway')]
    nx.draw_networkx_edges(G, pos, edgelist=edges_normal, edge_color='gray', width=1.5)

    # Gestrichelte Kanten zeichnen
    edges_dashed = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_dashed_highway')]
    nx.draw_networkx_edges(G, pos, edgelist=edges_dashed, edge_color='gray', width=1.5, style='--')

    node_degrees = dict(G.degree())
    node_color = ['red' if node_degrees[node] >= 3 else 'blue' for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_size=50, node_color=node_color)

    plt.title(f"Graph der Stadt: {city_name}")
    plt.xlabel("Längengrad")
    plt.ylabel("Breitengrad")
    plt.show()

plot_graph(city_name)

In [ ]:
node_degrees = dict(G.degree()) 

node_x = []
node_y = []
node_text = []  
node_color = [] 

for index, row in filtered_data_multiple.iterrows():
    if pd.notnull(row['longitude']) and pd.notnull(row['latitude']):
        node_x.append(row['longitude'])
        node_y.append(row['latitude'])
        
        hover_info = f"ID: {row['id']}<br>Tag Key: {row['tagkey']}<br>Tag Value: {row['tagvalue']}<br>Degree: {node_degrees[row['id']]}"
        node_text.append(hover_info)
        
        node_color.append(node_degrees[row['id']])

edge_x = []
edge_y = []
dashed_edge_x = []
dashed_edge_y = []
edge_street_names = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]

    # Überprüfen, ob der Rand highway != 'residual' ist
    is_dashed_highway = False
    street_name = ""
    for street in ways_filter.itertuples():
        if edge[0] in street.node_ids and edge[1] in street.node_ids:
            street_name = street.tagvalue
            # Highway-Kanten, die nicht 'residual' sind, werden gestrichelt
            if street.tagkey == "highway" and street.tagvalue not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']:
                is_dashed_highway = True
            break

    if is_dashed_highway:
        dashed_edge_x.extend([x0, x1, None])
        dashed_edge_y.extend([y0, y1, None])
    else:
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Berechnung der Mittelpunkte für die Straßennamen
    midpoint_x = (x0 + x1) / 2
    midpoint_y = (y0 + y1) / 2
    edge_street_names.append((midpoint_x, midpoint_y, street_name))


# Plot für durchgezogene Kanten
solid_edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Plot für gestrichelte Kanten (highways)
dashed_edge_trace = go.Scatter(
    x=dashed_edge_x, y=dashed_edge_y,
    line=dict(width=0.5, color='#888', dash='dash'),
    hoverinfo='none',
    mode='lines'
)

# Plot für Knoten
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        color=node_color,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Knoten Verbindungen',
            xanchor='left',
            titleside='right'
        )),
    text=node_text
)

# Hinzufügen der Straßennamen zwischen den Knoten
street_name_trace = go.Scatter(
    x=[name[0] for name in edge_street_names],
    y=[name[1] for name in edge_street_names],
    text=[name[2] for name in edge_street_names],
    mode='text',
    textfont=dict(
        size=12,
        color='darkblue'
    ),
    hoverinfo='text'
)

layout = go.Layout(
    title=city_name,
    titlefont=dict(size=16),
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=0),
    annotations=[dict(
        text=title,
        showarrow=False,
        xref="paper", yref="paper",
        x=0.005, y=-0.002)],
    xaxis=dict(showgrid=False, zeroline=False, visible=False),
    yaxis=dict(showgrid=False, zeroline=False, visible=False)
)

fig = go.Figure(data=[solid_edge_trace, dashed_edge_trace, node_trace], layout=layout)

fig.show()

### Straßenlänge berechnen und im Diagramm darstellen

In [ ]:
# Funktion zur Berechnung der Straßlänge
def calculate_full_road_length(way_row, df_nodes):
    node_ids = way_row['node_ids']
    total_length = 0

    for i in range(len(node_ids) - 1):
        first_node = df_nodes[df_nodes['id'] == node_ids[i]]
        second_node = df_nodes[df_nodes['id'] == node_ids[i + 1]]

        if first_node.empty or second_node.empty:
            return None

        total_length += haversine(
            first_node['longitude'].values[0], first_node['latitude'].values[0],
            second_node['longitude'].values[0], second_node['latitude'].values[0]
        )

    return total_length

# Funktion zur Verarbeitung der JSON-Dateien
def process_city_file(json_file_path, city_name):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    nodes_data = pd.DataFrame(data['nodes'])
    ways_data = pd.DataFrame(data['ways'])

    road_lengths = []

    for _, way_row in ways_data.iterrows():
        length = calculate_full_road_length(way_row, nodes_data)
        if length is not None:
            road_lengths.append({'city': city_name, 'way_id': way_row['id'], 'length': length})

    return pd.DataFrame(road_lengths)

# Dateien verarbeiten
aschaffenburg_file = "City_data/Aschaffenburg.json"
ringheim_file = "City_data/Ringheim.json"

df_aschaffenburg = process_city_file(aschaffenburg_file, "Aschaffenburg")
df_ringheim = process_city_file(ringheim_file, "Ringheim")

# Daten zusammenführen
df_combined = pd.concat([df_aschaffenburg, df_ringheim], ignore_index=True)

# Diagramme erstellen
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Boxplots
df_combined.boxplot(column='length', by='city', ax=axes[0, 0])
axes[0, 0].set_title("Boxplot der Straßenlängen")
axes[0, 0].set_xlabel("Stadt")
axes[0, 0].set_ylabel("Straßenlänge (m)")

# Histogramme
cities = ["Aschaffenburg", "Ringheim"]
for ax, city in zip(axes[1, :], cities):
    df_city = df_combined[df_combined['city'] == city]
    ax.hist(df_city['length'], bins=20, color='skyblue', edgecolor='black')
    ax.set_title(f"Histogramm der Straßenlängen in {city}")
    ax.set_xlabel("Straßenlänge (m)")
    ax.set_ylabel("Häufigkeit")

# Gesamttitel und Layout
plt.suptitle("Straßenlängenanalyse: Boxplot und Histogramm")
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


### Diagramm um den Grad der Knoten

In [ ]:
# Funktion zur Berechnung der Knotengrade
def create_graph_from_ways(ways_data):
    G = nx.Graph()
    for _, way_row in ways_data.iterrows():
        node_ids = way_row['node_ids']
        edges = [(node_ids[i], node_ids[i + 1]) for i in range(len(node_ids) - 1)]
        G.add_edges_from(edges)
    return G

def plot_degree_histogram(node_degrees, city_name, ax):
    degrees = list(node_degrees.values())
    unique_degrees, counts = np.unique(degrees, return_counts=True)

    ax.bar(unique_degrees, counts, color='skyblue', edgecolor='black')
    ax.set_title(f"Degree Histogram of {city_name}")
    ax.set_xlabel("Degree")
    ax.set_ylabel("# of Nodes")
    ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

G_aschaffenburg = create_graph_from_ways(df_aschaffenburg)
G_ringheim = create_graph_from_ways(df_ringheim)

node_degrees_aschaffenburg = dict(G_aschaffenburg.degree())
node_degrees_ringheim = dict(G_ringheim.degree())

# Histogramme erstellen
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

plot_degree_histogram(node_degrees_aschaffenburg, "Aschaffenburg", axes[0])
plot_degree_histogram(node_degrees_ringheim, "Ringheim", axes[1])

plt.suptitle("Knotengrad-Histogramme")
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

### Diagramm-Vergleich

In [ ]:
# Plot with Degree Diagrams
image_folder = 'Degree of various places'
images = [file for file in os.listdir(image_folder) if file.endswith(('png'))]

# Number of images
num_images = len(images)

cols = 3
rows = num_images // cols + (num_images % cols > 0)

fig, axs = plt.subplots(rows, cols, figsize=(15, 5 * rows))

for i, img_name in enumerate(images):
    img_path = os.path.join(image_folder, img_name)
    img = mpimg.imread(img_path)

    ax = axs[i // cols, i % cols] if rows > 1 else axs[i % cols]
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(img_name)

for j in range(i + 1, rows * cols):
    fig.delaxes(axs.flatten()[j])

plt.tight_layout()
plt.show()